In [5]:
data_root = '/Users/mallem/Downloads/'
caffe_root = '/Users/mallem/dev/caffe/'

import numpy as np
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe
import os
if not os.path.isfile(caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print("Downloading pre-trained CaffeNet model...")
    !/Users/mallem/dev/caffe/scripts/download_model_binary.py /Users/mallem/dev/caffe/models/bvlc_reference_caffenet
print("Downloaded pre-trained CaffeNet model...")    

Downloaded pre-trained CaffeNet model...


In [2]:
def extract_features(images, layer = 'fc7'):
    net = caffe.Net(caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt',
                caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
                caffe.TEST)
    
    # input preprocessing: 'data' is the name of the input blob == net.inputs[0]
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) 
    transformer.set_raw_scale('data', 255)  
    transformer.set_channel_swap('data', (2,1,0)) 

    num_images= len(images)
    net.blobs['data'].reshape(num_images,3,227,227)
    net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data',caffe.io.load_image(x)), images)
    out = net.forward()

    return net.blobs[layer].data        

In [10]:
f = h5py.File(data_root+'train_image_fc7features.h5','r')
print 'train_image_features.h5:'
for key in f.keys():
    print key, f[key].shape
    
print "\nA photo:", f['photo_id'][0]
print "Its feature vector (first 50-dim): ", f['feature'][0][0:50], " ..."
f.close()

train_image_features.h5:
feature (234842, 4096)
photo_id (234842,)

A photo: /Users/mallem/Downloads/train_photos/204149.jpg
Its feature vector (first 50-dim):  [ 0.          0.          0.          0.          0.          0.10604793
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  2.15204549  0.          2.70966959  0.          0.          1.56889081
  0.          0.          0.          0.55355042  0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.        ]  ...


In [3]:
# extract image features and save it to .h5
 
import h5py
f = h5py.File(data_root+'train_image_fc7features.h5','w')
filenames = f.create_dataset('photo_id',(0,), maxshape=(None,),dtype='|S54')
feature = f.create_dataset('feature',(0,4096), maxshape = (None,4096))
f.close()

import pandas as pd 
train_photos = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_folder = data_root+'train_photos/'
train_images = [os.path.join(train_folder, str(x)+'.jpg') for x in train_photos['photo_id']]  # get full filename

num_train = len(train_images)
print "Number of training images: ", num_train
batch_size = 500

# Training Images
for i in range(0, num_train, batch_size): 
    images = train_images[i: min(i+batch_size, num_train)]
    features = extract_features(images, layer='fc7')
    num_done = i+features.shape[0]
    f= h5py.File(data_root+'train_image_fc7features.h5','r+')
    f['photo_id'].resize((num_done,))
    f['photo_id'][i: num_done] = np.array(images)
    f['feature'].resize((num_done,features.shape[1]))
    f['feature'][i: num_done, :] = features
    f.close()
    if num_done%10000==0 or num_done==num_train:
        print "Train images processed: ", num_done

Number of training images:  234842
Train images processed:  10000
Train images processed:  20000
Train images processed:  30000
Train images processed:  40000
Train images processed:  50000
Train images processed:  60000
Train images processed:  70000
Train images processed:  80000
Train images processed:  90000
Train images processed:  100000
Train images processed:  110000
Train images processed:  120000
Train images processed:  130000
Train images processed:  140000
Train images processed:  150000
Train images processed:  160000
Train images processed:  170000
Train images processed:  180000
Train images processed:  190000
Train images processed:  200000
Train images processed:  210000
Train images processed:  220000
Train images processed:  230000
Train images processed:  234842
